# **Measure Organelle Interactions**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

In notebooks 2.1 through 2.4, we will go over the implementation of `infer-subc` quantification methods (explained in detail in the `method_...` notebooks) to assess the morphology, interactions, and distribution of organelles at the single-cell level. 

### 📍 **Purpose**
This notebook can be used to measure the morphology and distribution of `organelle interactions` from one or more cells. It includes options to:
1. 🦠 Quantify *one or more organelle interaction types* from <ins>ONE CELL</ins>
2. 🧪 Batch process the quantification of *all possible organelle interaction types* from *multiple cells* for a <ins>SINGLE EXPERIMENT</ins>
3. 🧮 Summarize organelle interaction metrics *per cell* across <INS>ONE OR MORE EXPERIMENTS</ins>

### 🍃 **Biological Relevance - Organelle Interactions**
Intracellular organelles do not exist independently of one another. In recent years, the existance and function of organelle contact site, regions of close apposition between membrane bound organelles has been recognized. They facilitate protein, lipid, and metabolite transport, coordinate organelle trafficking and function, and are involved in many cellular pathways and functions [[1](https://www.cell.com/cell/pdf/S0092-8674(23)01328-4.pdf)]. 

The distance between membranes at contact sites is between 30-80 nm depending on the types of organelles involved. This distance is not resolvable using standard confocal microscopy images [[2](https://zeiss-campus.magnet.fsu.edu/articles/basics/resolution.html)]. However, interactions between organelle which can include organelle contact sites can be estimated through overlap in label localization in confocal microscopy images.

Here, we will use the overlapping area between two or more organelle objects as putative interaction sites. The current notebook is only formatted to collect information about pairwise interactions between organelles, but this will be expanded in future versions to include all possible n-way interaction types.

**`Organelle interaction sites`**: regions of overlap between two or more organelles

These sites can then be measured for features such as number, size, and shape utilizing the `get_morpholgy_metrics()` function outlined in [method_morphology](method_morphology.ipynb) notebook and/or measurements of subcellular distribution utilizing the `get_distribution()` function outlined in [method_distribution](method_distribution.ipynb) notebook.

*You can learn more about how interaction sites are created within infer-subc in the [method_interactions](method_interactions.ipynb) notebook.*

-----

## 🗂️ **Table of Contents**
The following sections are included in this notebook:

**IMPORTS AND LOAD IMAGE**

**EXPLANATION OF STEPS** - This section serves as *expository examples* of the functions used to quantify, batch process, and summarize organelle morphology.

🦠 **Quantify *one or more organelle interaction types* from <ins>ONE CELL</ins>**
- **`STEP 1`** - Apply cell mask for single cell analysis
- **`STEP 2`** - Loop through the list of organelles to quantify the morphology of each
- **`STEP 3`** - Combine all of the tables together and add column
- **`DEFINE`** - The get_organelle_morph() function

🧪 **Batch process the quantification of *all possible organelle interaction types* from *multiple cells* for a <ins>SINGLE EXPERIMENT</ins>**
- **`STEP 1`** - List images and segmentations to be collected for each
- **`STEP 2`** - Loop through the list of images and perform the morphology quantification on all organelles
- **`STEP 3`** - Combine all of the tables together and create/store the csv file
- **`DEFINE`** - The batch_process_org_morph() function

🧮 **Summarize organelle interaction metrics *per cell* across <INS>ONE OR MORE EXPERIMENTS</ins>**
- **`STEP 1`** - Get the orgnaelle morphology .csv files
- **`STEP 2`** - Summarize the mean, median, and standard deviation of each feature per cell
- **`STEP 3`** - Calculate additional metrics
- **`STEP 4`** - Unstack the organelle names, fill NA values with 0, and save file
- **`DEFINE`** - The batch_org_morph_summary_stats() function

**EXECUTE QUANTIFICATION** - Once you understand how the functions work, this section can be used to quantify your data in a quick and easy way.
- **`STEP 1`:** 🧪 **Batch process the quantification of *all possible organelle interaction types* from *multiple cells* for a <ins>SINGLE EXPERIMENT</ins>**
- **`STEP 2`:** 🧮 **Summarize organelle interaction metrics *per cell* across <INS>ONE OR MORE EXPERIMENTS</ins>**

-----
---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [ ]:
from typing import List, Union
from pathlib import Path
import os
import time
import warnings

from infer_subc.core.img import *

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from infer_subc.utils.stats import get_morphology_metrics, get_org_morphology, batch_process_org_morph, batch_org_morph_summary_stats
from infer_subc.utils.batch import list_image_files, find_segmentation_tiff_files
from infer_subc.core.file_io import read_czi_image, read_tiff_image

pd.set_option('display.max_columns', None)

#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [ ]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_two"
seg_data_path = data_root_path / "out_two"
quant_data_path = data_root_path / "quant_two"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi
1,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a48hrs-Ctrl + oleic acid_01_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [ ]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, `suffix_separator`, and `mask_name`.

In [ ]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-20230426_test_"
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

# colors = ["red", "bop orange", "yellow", "green", "blue", "cyan", "magenta", "bop purple"]
for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez

09-Feb-25 13:18:45 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x661+1530+213 (frame: 1104x708+1521+175) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 905x831+1528+206 (frame: 923x878+1519+168) margins: 9, 38, 9, 9 minimum size: 385x519 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=403,566 maxtrack=0,0)


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_two/a24hrs-Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24h

------
-----
## **EXPLANATION OF STEPS**

-----
### 🦠 **Quantify *one or more organelle interaction types* from <ins>ONE CELL</ins>**

#### **`STEP 1` - Create region of overlap between organelle 'a' and organelle 'b'**